# 检索召回

将文本转换成向量之后，它们的空间距离越近，就认为它们是越相似的，我们来召回相似语句。

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import numpy as np

load_dotenv()
client = OpenAI()

In [2]:
query = ['感冒吃什么药效果好？可以吃阿莫西林吗？']  # 查询语句

# 向量化
query_resp = client.embeddings.create(
    model="m3e",
    input=query
)

query_vec = np.array(query_resp.data[0].embedding)  # 查询语句向量，由于后续要对向量进行计算，所以先预先转为np格式向量

In [3]:
query_vec

array([ 0.03853967,  0.01111819, -0.01225554, ...,  0.01528172,
       -0.01386917, -0.00429631])

In [4]:
# 备选文本向量化
question_texts = [
    '什么叫感冒? / 感冒是一种什么病？',
    '感冒一般是由什么引起的？/ 什么会导致感冒？',
    '感冒会有哪些症状？/ 感冒有哪些临床表现？',
    '感冒吃什么药好得快？/ 感冒怎么治？',
    '得了感冒去医院挂什么科室的号？',
    '感冒要怎么预防？',
    '感冒换着有什么禁忌？/ 感冒不能吃什么？',
    '感冒要做哪些检查？',
    '感冒能治好吗？/ 感冒治好的几率有多大？',
    '感冒的并发症有哪些？',
    '阿莫西林能治那些病？'
]

question_resp = client.embeddings.create(
    model="m3e",
    input=question_texts
)

question_vecs = []
for item in question_resp.data:
    question_vecs.append(np.array(item.embedding))
    
len(question_vecs)  # 一共11个备选问题

11

In [5]:
question_vecs

[array([ 0.03816124,  0.01146382, -0.01184995, ...,  0.02825603,
        -0.00140581, -0.00344242]),
 array([ 0.03908591,  0.01380969, -0.00870894, ...,  0.029853  ,
        -0.0023192 , -0.00234774]),
 array([ 0.03883881,  0.00198416, -0.02499154, ...,  0.00466784,
        -0.00071755, -0.00011842]),
 array([ 0.03708535, -0.01513652, -0.00726908, ..., -0.01910228,
         0.01162669,  0.0113326 ]),
 array([ 0.03106184,  0.03302555,  0.00379826, ...,  0.04060166,
        -0.03795596,  0.01676303]),
 array([ 0.03532001, -0.02236142, -0.01250461, ..., -0.04777015,
         0.00123236,  0.02102368]),
 array([ 0.03905035, -0.01533284, -0.01019276, ..., -0.02456447,
         0.00025354,  0.01260522]),
 array([ 0.03815147,  0.0109698 , -0.01118307, ...,  0.02219287,
        -0.00675634,  0.00055178]),
 array([ 4.00703177e-02, -5.11546899e-03, -1.35153290e-02, ...,
        -9.16950591e-03,  1.19345973e-03,  9.42188635e-05]),
 array([ 0.03884551, -0.01269325, -0.03441332, ..., -0.0281163 ,
  

现在的场景为：用户输入了query，我们需要将query和我们预先准备的question_texts作比对，找出最相似的前3条question_texts。

如下的代码需要注意，这里为了简单，我们采用了欧式距离，即

\[
d = \sqrt{(x_2 - x_1)^2 + (y_2 - y_1)^2}
\]

距离越小，相似度越高。

其次，query_vec - question_vecs利用到了广播机制，因为前者只有1个向量，后者有11个向量，所以np底层会依次做计算，即广播机制。

axis=1即按列操作，从左往右。

例子：

- 查询向量 \( \mathbf{q} = [1, 2, 3] \)
- 问题向量组 \( \mathbf{Q} = \begin{bmatrix} 1 & 2 & 3 \\ 4 & 5 & 6 \\ 7 & 8 & 9 \end{bmatrix} \)

1. **计算差值矩阵**：
   \[
   \mathbf{D} = \mathbf{q} - \mathbf{Q} = \begin{bmatrix} 1 - 1 & 2 - 2 & 3 - 3 \\ 1 - 4 & 2 - 5 & 3 - 6 \\ 1 - 7 & 2 - 8 & 3 - 9 \end{bmatrix} = \begin{bmatrix} 0 & 0 & 0 \\ -3 & -3 & -3 \\ -6 & -6 & -6 \end{bmatrix}
   \]

2. **计算每个差值向量的平方**：
   \[
   \mathbf{D}^2 = \begin{bmatrix} 0^2 & 0^2 & 0^2 \\ (-3)^2 & (-3)^2 & (-3)^2 \\ (-6)^2 & (-6)^2 & (-6)^2 \end{bmatrix} = \begin{bmatrix} 0 & 0 & 0 \\ 9 & 9 & 9 \\ 36 & 36 & 36 \end{bmatrix}
   \]

3. **计算每个差值向量的平方和**：
   \[
   \text{Sum of squares} = \begin{bmatrix} 0 + 0 + 0 \\ 9 + 9 + 9 \\ 36 + 36 + 36 \end{bmatrix} = \begin{bmatrix} 0 \\ 27 \\ 108 \end{bmatrix}
   \]

4. **计算每个平方和的平方根（即 L2 距离）**：
   \[
   \text{L2 distances} = \begin{bmatrix} \sqrt{0} \\ \sqrt{27} \\ \sqrt{108} \end{bmatrix} = \begin{bmatrix} 0 \\ 5.196 \\ 10.392 \end{bmatrix}
   \]

查询向量 \( \mathbf{q} \) 与问题向量组 \( \mathbf{Q} \) 中每个向量之间的 L2 距离为：
\[
\text{L2 distances} = \begin{bmatrix} 0 \\ 5.196 \\ 10.392 \end{bmatrix}
\]

In [6]:
import numpy as np

l2_distances = np.linalg.norm(query_vec - question_vecs, axis=1)  

closest_indices = np.argsort(l2_distances)[:3]  # 从小到大排列，取前三个
closest_indices   # 最像的问题是第3，10，0个问题

array([ 3, 10,  0], dtype=int64)

In [7]:
for idx in closest_indices:
    print(question_texts[idx])

感冒吃什么药好得快？/ 感冒怎么治？
阿莫西林能治那些病？
什么叫感冒? / 感冒是一种什么病？
